In [1]:
import pandas as pd
# https://www.avera.org/patients-visitors/price-transparency/

In [2]:
import csv
import subprocess
from urllib.parse import urlparse
import os

from dateutil.parser import parse as parse_datetime
import pandas as pd

In [3]:
TARGET_COLUMNS = [ 
    'hospital_id',
    #'row_id',
    'line_type',
    'description',
    'rev_code',
    'local_code',
    'code',
    'ms_drg',
    'apr_drg',
    'eapg',
    'hcpcs_cpt',
    'modifiers',
    'thru',
    'apc',
    'icd',
    'ndc',
    'drug_hcpcs_multiplier',
    'drug_quantity',
    'drug_unit_of_measurement',
    'drug_type_of_measurement',
    'billing_class',
    'setting',
    'payer_category',
    'payer_name',
    'plan_name',
    'standard_charge',
    'standard_charge_percent',
    'contracting_method',
    'additional_generic_notes',
    'additional_payer_specific_notes'
]

In [4]:
ccn = "160124"
url = "https://www.avera.org/app/files/public/e920ec8c-5f0b-480e-b0dd-8644ed571c8d/426037582_lakes-regional-hospital_standardcharges.xlsx"
filename = url.split("/")[-1].replace("%20", " ")
ein = filename.split("_")[0]
ein = ein[:2] + "-" + ein[2:]

In [5]:
import httpx
import os

def create_session():
    session = httpx.Client(http2=True)
    
    session.headers = {
        'authority': 'www.avera.org',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
        'cache-control': 'no-cache',
        'pragma': 'no-cache',
        'sec-ch-ua': '"Chromium";v="112", "Google Chrome";v="112", "Not:A-Brand";v="99"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"macOS"',
        'sec-fetch-dest': 'document',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-site': 'none',
        'sec-fetch-user': '?1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36',
    }
    
    resp = session.get("https://www.avera.org/patients-visitors/price-transparency/")
    print(resp.url)
    
    return session

def download_file(session, url, filename):
    if os.path.isfile(filename):
        print("{} already downloaded - skipping".format(filename))
        return
    
    resp = session.get(url)
    print(resp.url)
    
    out_f = open(filename, "wb")
    out_f.write(resp.content)
    out_f.close()
    
session = create_session()
download_file(session, url, filename)

https://www.avera.org/patients-visitors/price-transparency/
426037582_lakes-regional-hospital_standardcharges.xlsx already downloaded - skipping


In [6]:
df_in = pd.read_excel(filename, skiprows=1, sheet_name='CDM')
df_in

,Facility Name,CPT,Charge Code,Gross Charge,Description,Hcpcs,Revcode,BCBS Commercial Negotiated Rate,Wellmark PPO Negotiated Rate,HealthPartners Commercial Negotiated Rate,PreferredOne PPO Negotiated Rate,Medica Commercial Negotiated Rate,UnitedHealthcare Commercial Negotiated Rate,Avera Health Plan Commercial Negotiated Rate,Discounted Cash Price,Minimum Negotiated Rate,Maximum Negotiated Rate
0,Lakes Regional Hospital,90647,19S0727,0.01,Hib PRP-OMP 3 dose,NaN,636,NaN,NaN,0.0093,0.0094,0.008,0.0085,0.0095,0.009,0.008,0.0095
1,Lakes Regional Hospital,90670,19S0749,0.01,Pneumococcal (PCV13),NaN,636,NaN,NaN,0.0093,0.0094,0.008,0.0085,0.0095,0.009,0.008,0.0095
2,Lakes Regional Hospital,90680,19S0752,0.01,Rotavirus (RV5) 3 dose,NaN,636,NaN,NaN,0.0093,0.0094,0.008,0.0085,0.0095,0.009,0.008,0.0095
3,Lakes Regional Hospital,90723,19S0773,0.01,DTaP-HepB-IPV IM,NaN,636,NaN,NaN,0.0093,0.0094,0.008,0.0085,0.0095,0.009,0.008,0.0095
4,Lakes Regional Hospital,77063,21L2364,158.00,Breast Tomosynthesis BI,NaN,403,NaN,NaN,146.9400,149.1520,126.558,134.3000,150.1000,142.200,126.558,150.1000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3093,Lakes Regional Hospital,29515,9680845,165.00,Apply Short Leg Splint RT,NaN,981,NaN,NaN,153.4500,155.7600,132.165,140.2500,156.7500,148.500,132.165,156.7500
3094,Lakes Regional Hospital,30901,9680909,335.00,Control Anterior Hemorrhage RT,NaN,981,NaN,NaN,311.5500,316.2400,268.335,284.7500,318.2500,301.500,268.335,318.2500
3095,Lakes Regional Hospital,36558,9681056,815.00,Insert Tunneled CV Catheter RT,NaN,981,NaN,NaN,757.9500,769.3600,652.815,692.7500,774.2500,733.500,652.815,774.2500
3096,Lakes Regional Hospital,69200,9681952,208.00,Remove Ear Foreign Body RT,NaN,981,NaN,NaN,193.4400,196.3520,166.608,176.8000,197.6000,187.200,166.608,197.6000


In [7]:
def payer_category_from_payer(payer):
    if payer == 'Gross Charge':
        return 'gross'
    elif payer == 'Discounted Cash Price':
        return 'cash'
    elif payer == 'Minimum Negotiated Rate':
        return 'min'
    elif payer == 'Maximum Negotiated Rate':
        return 'max'
    
    return 'payer'

def convert_dataframe(df_in, ccn):
    df_mid = pd.DataFrame(df_in)

    df_mid = df_mid.rename(columns={
        'CPT': 'cpt',
        'Charge Code': 'local_code',
        'Description': 'description',
        'Hcpcs': 'hcpcs',
        'Revcode': 'rev_code'
    })

    columns = df_mid.columns.to_list()
    columns.pop(columns.index('Gross Charge'))
    df_mid = df_mid[columns + ['Gross Charge']]

    del df_mid['Facility Name']

    money_columns = df_mid.columns[5:]
    remaining_columns = df_mid.columns.to_list()[:5]
    df_mid = pd.melt(df_mid, id_vars=remaining_columns, var_name='payer_name', value_name='standard_charge')
    
    def join_hcpcs_cpt_fields(row):
        if row['cpt'] is not None:
            return row['cpt']
        
        if row['hcpcs'] is not None:
            return row['hcpcs']
        
        return None
    
    df_mid.loc[df_mid['cpt'].isnull(), 'cpt'] = None
    df_mid.loc[df_mid['hcpcs'].isnull(), 'hcpcs'] = None
    
    df_mid['hcpcs_cpt'] = df_mid.apply(join_hcpcs_cpt_fields, axis=1)
    del df_mid['cpt']
    del df_mid['hcpcs']

    df_mid['payer_category'] = df_mid['payer_name'].apply(payer_category_from_payer)

    df_mid['hospital_id'] = ccn
    df_mid['line_type'] = None
    df_mid['rev_code'] = df_mid['rev_code'].fillna('')
    df_mid['rev_code'] = df_mid['rev_code'].astype(str)
    df_mid['rev_code'] = df_mid['rev_code'].str[:4] # 0964an
    df_mid['rev_code'] = df_mid['rev_code'].apply(
        lambda rev_code: rev_code.zfill(4) if len(rev_code) > 0 and len(rev_code) < 4 else rev_code
    )
    df_mid.loc[df_mid['rev_code'] == '', 'rev_code'] = None
    df_mid['code'] = None
    df_mid['ms_drg'] = None
    df_mid['apr_drg'] = None
    df_mid['eapg'] = None
    df_mid['modifiers'] = ''
    df_mid['thru'] = None
    df_mid['apc'] = None
    df_mid['icd'] = None
    df_mid['ndc'] = None
    df_mid['drug_hcpcs_multiplier'] = None
    df_mid['drug_quantity'] = None
    df_mid['drug_unit_of_measurement'] = None
    df_mid['drug_type_of_measurement'] = None
    df_mid['billing_class'] = None
    df_mid['setting'] = None
    df_mid['plan_name'] = None
    df_mid['standard_charge_percent'] = None
    df_mid['contracting_method'] = None
    df_mid['additional_generic_notes'] = None
    df_mid['additional_payer_specific_notes'] = None
    
    df_mid = df_mid.dropna(subset=['standard_charge'])

    df_out = pd.DataFrame(df_mid[TARGET_COLUMNS])

    return df_out

In [8]:
df_out = convert_dataframe(df_in, ccn)
df_out

,hospital_id,line_type,description,rev_code,local_code,code,ms_drg,apr_drg,eapg,hcpcs_cpt,...,billing_class,setting,payer_category,payer_name,plan_name,standard_charge,standard_charge_percent,contracting_method,additional_generic_notes,additional_payer_specific_notes
6196,160124,None,Hib PRP-OMP 3 dose,0636,19S0727,None,None,None,None,90647,...,None,None,payer,HealthPartners Commercial Negotiated Rate,None,0.0093,None,None,None,None
6197,160124,None,Pneumococcal (PCV13),0636,19S0749,None,None,None,None,90670,...,None,None,payer,HealthPartners Commercial Negotiated Rate,None,0.0093,None,None,None,None
6198,160124,None,Rotavirus (RV5) 3 dose,0636,19S0752,None,None,None,None,90680,...,None,None,payer,HealthPartners Commercial Negotiated Rate,None,0.0093,None,None,None,None
6199,160124,None,DTaP-HepB-IPV IM,0636,19S0773,None,None,None,None,90723,...,None,None,payer,HealthPartners Commercial Negotiated Rate,None,0.0093,None,None,None,None
6200,160124,None,Breast Tomosynthesis BI,0403,21L2364,None,None,None,None,77063,...,None,None,payer,HealthPartners Commercial Negotiated Rate,None,146.9400,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34073,160124,None,Apply Short Leg Splint RT,0981,9680845,None,None,None,None,29515,...,None,None,gross,Gross Charge,None,165.0000,None,None,None,None
34074,160124,None,Control Anterior Hemorrhage RT,0981,9680909,None,None,None,None,30901,...,None,None,gross,Gross Charge,None,335.0000,None,None,None,None
34075,160124,None,Insert Tunneled CV Catheter RT,0981,9681056,None,None,None,None,36558,...,None,None,gross,Gross Charge,None,815.0000,None,None,None,None
34076,160124,None,Remove Ear Foreign Body RT,0981,9681952,None,None,None,None,69200,...,None,None,gross,Gross Charge,None,208.0000,None,None,None,None


In [9]:
df_out[df_out['hcpcs_cpt'] == 'G0379']

,hospital_id,line_type,description,rev_code,local_code,code,ms_drg,apr_drg,eapg,hcpcs_cpt,...,billing_class,setting,payer_category,payer_name,plan_name,standard_charge,standard_charge_percent,contracting_method,additional_generic_notes,additional_payer_specific_notes
6210,160124,None,Observation Assess Direct Adm,0762,0100201,None,None,None,None,G0379,...,None,None,payer,HealthPartners Commercial Negotiated Rate,None,91.140,None,None,None,None
9308,160124,None,Observation Assess Direct Adm,0762,0100201,None,None,None,None,G0379,...,None,None,payer,PreferredOne PPO Negotiated Rate,None,92.512,None,None,None,None
12406,160124,None,Observation Assess Direct Adm,0762,0100201,None,None,None,None,G0379,...,None,None,payer,Medica Commercial Negotiated Rate,None,78.498,None,None,None,None
15504,160124,None,Observation Assess Direct Adm,0762,0100201,None,None,None,None,G0379,...,None,None,payer,UnitedHealthcare Commercial Negotiated Rate,None,83.300,None,None,None,None
18602,160124,None,Observation Assess Direct Adm,0762,0100201,None,None,None,None,G0379,...,None,None,payer,Avera Health Plan Commercial Negotiated Rate,None,93.100,None,None,None,None
21700,160124,None,Observation Assess Direct Adm,0762,0100201,None,None,None,None,G0379,...,None,None,cash,Discounted Cash Price,None,88.200,None,None,None,None
24798,160124,None,Observation Assess Direct Adm,0762,0100201,None,None,None,None,G0379,...,None,None,min,Minimum Negotiated Rate,None,78.498,None,None,None,None
27896,160124,None,Observation Assess Direct Adm,0762,0100201,None,None,None,None,G0379,...,None,None,max,Maximum Negotiated Rate,None,93.100,None,None,None,None
30994,160124,None,Observation Assess Direct Adm,0762,0100201,None,None,None,None,G0379,...,None,None,gross,Gross Charge,None,98.000,None,None,None,None
